In [1]:
# %% [markdown]
# # Eliminación de Registros Duplicados en Archivos CSV
# 
# Este notebook identifica y elimina registros duplicados en los archivos CSV proporcionados.

# %%
import pandas as pd
from IPython.display import display, HTML
import os

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Directorio para guardar archivos limpios
os.makedirs('csv_limpios', exist_ok=True)

# %% [markdown]
# ## 1. Definición de Archivos y Estrategias de Limpieza

# %%
archivos = {
    'alumnos': {
        'path': 'csv/alumnos.csv',
        'columnas_duplicados': ['_id', 'id_alumno', 'curp', 'rfc'],
        'keep': 'first'  # Conservar el primer registro duplicado
    },
    'encuestas': {
        'path': 'csv/encuestas.csv',
        'columnas_duplicados': ['_id', 'correo', 'id_alumno'],
        'keep': 'first'
    },
    'users': {
        'path': 'csv/users.csv',
        'columnas_duplicados': ['_id', 'correo'],
        'keep': 'first'
    },
    'predicciones': {
        'path': 'csv/predicciones.csv',
        'columnas_duplicados': ['_id', 'id_encuesta', 'id_alumno'],
        'keep': 'first'
    },
    # Archivos que no requieren limpieza (solo para registro)
    'carreras': {'path': 'csv/carreras.csv'},
    'grupos': {'path': 'csv/grupos.csv'},
    'roles': {'path': 'csv/roles.csv'},
    'migrations': {'path': 'csv/migrations.csv'}
}

# %% [markdown]
# ## 2. Función para Eliminar Duplicados

# %%
def limpiar_duplicados(nombre_archivo, config):
    print(f"\n{'='*80}")
    print(f"🧹 PROCESANDO: {nombre_archivo.upper()}")
    print('='*80)
    
    try:
        # Leer archivo
        df = pd.read_csv(config['path'])
        original_rows = len(df)
        print(f"📊 Filas originales: {original_rows:,}")
        
        # Verificar si necesita limpieza
        if 'columnas_duplicados' not in config:
            print("✅ Archivo no requiere limpieza de duplicados")
            return None
        
        # %% [markdown]
        ### Análisis de Duplicados
        
        # %%
        print("\n🔍 Análisis de duplicados por columnas:")
        
        resultados = []
        for col in config['columnas_duplicados']:
            if col in df.columns:
                dups = df.duplicated(subset=[col], keep=False).sum()
                porcentaje = (dups / original_rows) * 100
                resultados.append([col, dups, f"{porcentaje:.2f}%"])
                print(f" - Columna '{col}': {dups} duplicados ({porcentaje:.2f}%)")
        
        # Mostrar resumen
        if resultados:
            display(HTML(
                pd.DataFrame(resultados, columns=['Columna', 'Duplicados', 'Porcentaje'])
                .style.set_properties(**{'background-color': '#f8f9fa', 'border': '1px solid #ddd'})
                .to_html()
            ))
        
        # %% [markdown]
        ### Eliminación de Duplicados
        
        # %%
        print("\n🧹 Eliminando duplicados...")
        
        # Eliminar duplicados exactos (filas completas)
        df_clean = df.drop_duplicates(keep=config['keep'])
        
        # Eliminar duplicados por columnas clave
        for col in config['columnas_duplicados']:
            if col in df.columns:
                df_clean = df_clean.drop_duplicates(subset=[col], keep=config['keep'])
        
        nuevos_registros = len(df_clean)
        eliminados = original_rows - nuevos_registros
        
        print(f"✅ Filas después de limpieza: {nuevos_registros:,}")
        print(f"🗑️ Registros eliminados: {eliminados} ({eliminados/original_rows*100:.2f}%)")
        
        # %% [markdown]
        ### Guardar Archivo Limpio
        
        # %%
        output_path = f"csv_limpios/{nombre_archivo}_limpio.csv"
        df_clean.to_csv(output_path, index=False)
        print(f"\n💾 Archivo limpio guardado en: {output_path}")
        
        return df_clean
    
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        return None

# %% [markdown]
# ## 3. Procesar Todos los Archivos

# %%
for nombre_archivo, config in archivos.items():
    limpiar_duplicados(nombre_archivo, config)

# %% [markdown]
# ## 4. Función para Comparar Archivos

# %%
def comparar_archivos(nombre_archivo):
    original_path = archivos[nombre_archivo]['path']
    limpio_path = f'csv_limpios/{nombre_archivo}_limpio.csv'
    
    if not os.path.exists(limpio_path):
        print(f"No existe archivo limpio para {nombre_archivo}")
        return
    
    df_original = pd.read_csv(original_path)
    df_limpio = pd.read_csv(limpio_path)
    
    print(f"\n🔎 COMPARACIÓN: {nombre_archivo.upper()}")
    print('='*60)
    print(f"Original: {len(df_original):,} registros")
    print(f"Limpio: {len(df_limpio):,} registros")
    print(f"Registros eliminados: {len(df_original)-len(df_limpio):,}")
    
    # Mostrar algunos registros eliminados
    duplicados = df_original[~df_original.isin(df_limpio)].dropna(how='all')
    if not duplicados.empty:
        print("\n🗑️ Ejemplo de registros eliminados:")
        display(HTML(duplicados.head(3).to_html()))


🧹 PROCESANDO: ALUMNOS
📊 Filas originales: 13

🔍 Análisis de duplicados por columnas:
 - Columna '_id': 0 duplicados (0.00%)
 - Columna 'id_alumno': 6 duplicados (46.15%)
 - Columna 'curp': 0 duplicados (0.00%)
 - Columna 'rfc': 0 duplicados (0.00%)


,Columna,Duplicados,Porcentaje
0,_id,0,0.00%
1,id_alumno,6,46.15%
2,curp,0,0.00%
3,rfc,0,0.00%



🧹 Eliminando duplicados...
✅ Filas después de limpieza: 8
🗑️ Registros eliminados: 5 (38.46%)

💾 Archivo limpio guardado en: csv_limpios/alumnos_limpio.csv

🧹 PROCESANDO: ENCUESTAS
📊 Filas originales: 15

🔍 Análisis de duplicados por columnas:
 - Columna '_id': 0 duplicados (0.00%)
 - Columna 'correo': 12 duplicados (80.00%)
 - Columna 'id_alumno': 9 duplicados (60.00%)


,Columna,Duplicados,Porcentaje
0,_id,0,0.00%
1,correo,12,80.00%
2,id_alumno,9,60.00%



🧹 Eliminando duplicados...
✅ Filas después de limpieza: 1
🗑️ Registros eliminados: 14 (93.33%)

💾 Archivo limpio guardado en: csv_limpios/encuestas_limpio.csv

🧹 PROCESANDO: USERS
📊 Filas originales: 13

🔍 Análisis de duplicados por columnas:
 - Columna '_id': 0 duplicados (0.00%)
 - Columna 'correo': 0 duplicados (0.00%)


,Columna,Duplicados,Porcentaje
0,_id,0,0.00%
1,correo,0,0.00%



🧹 Eliminando duplicados...
✅ Filas después de limpieza: 13
🗑️ Registros eliminados: 0 (0.00%)

💾 Archivo limpio guardado en: csv_limpios/users_limpio.csv

🧹 PROCESANDO: PREDICCIONES
📊 Filas originales: 43

🔍 Análisis de duplicados por columnas:
 - Columna '_id': 0 duplicados (0.00%)
 - Columna 'id_encuesta': 42 duplicados (97.67%)
 - Columna 'id_alumno': 42 duplicados (97.67%)


,Columna,Duplicados,Porcentaje
0,_id,0,0.00%
1,id_encuesta,42,97.67%
2,id_alumno,42,97.67%



🧹 Eliminando duplicados...
✅ Filas después de limpieza: 5
🗑️ Registros eliminados: 38 (88.37%)

💾 Archivo limpio guardado en: csv_limpios/predicciones_limpio.csv

🧹 PROCESANDO: CARRERAS
📊 Filas originales: 2
✅ Archivo no requiere limpieza de duplicados

🧹 PROCESANDO: GRUPOS
📊 Filas originales: 6
✅ Archivo no requiere limpieza de duplicados

🧹 PROCESANDO: ROLES
📊 Filas originales: 2
✅ Archivo no requiere limpieza de duplicados

🧹 PROCESANDO: MIGRATIONS
📊 Filas originales: 3
✅ Archivo no requiere limpieza de duplicados
